In [26]:
import sys

sys.path.append("../")

In [27]:
import requests
import time
import pandas as pd
import numpy as np
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from stats_arb.tests import adf_test, kpss_test, cal_half_life, pp_test
from datetime import datetime, timedelta
from ta.volatility import BollingerBands

import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams["figure.figsize"] = (10, 6) # (w, h)
plt.ioff()

## CONFIG

In [42]:
API_BASE = 'https://fapi.binance.com/fapi/v1/'
# DATA_PATH = '/mnt/c/Users/vmodg/source/repos/BeanRepos/crypto-pair-trading/data/'
TIMEFRAME = '1d'
lookback = 1000

# TIMEFRAME = '1h'
# lookback = 30*4

# TIMEFRAME = '15m'
# lookback = 30*2


DATA_PATH = f'/mnt/d/Working/PersonalProjects/Trading/trading-agent/crypto-pair-trading/data/crypto/'


In [43]:


LABELS = [
    'open_time',
    'open',
    'high',
    'low',
    'close',
    'volume',
    'close_time',
    'quote_asset_volume',
    'number_of_trades',
    'taker_buy_base_asset_volume',
    'taker_buy_quote_asset_volume',
    'ignore'
]

DROP_COLUMNS=[
    'close_time',
    'quote_asset_volume',
    'number_of_trades',
    'taker_buy_base_asset_volume',
    'taker_buy_quote_asset_volume',
    'ignore'
]


def get_batch(symbol, interval='1m', start_time=0, limit=400):
    """Use a GET request to retrieve a batch of candlesticks. Process the JSON into a pandas
    dataframe and return it. If not successful, return an empty dataframe.
    """

    params = {
        'symbol': symbol,
        'interval': interval,
        'startTime': start_time,
        'limit': limit
    }
    try:
        # timeout should also be given as a parameter to the function
        response = requests.get(f'{API_BASE}klines', params, timeout=30)
    except requests.exceptions.ConnectionError:
        print('Connection error, Cooling down for 5 mins...')
        time.sleep(15)
        return get_batch(symbol, interval, start_time, limit)

    except requests.exceptions.Timeout:
        print('Timeout, Cooling down for 5 min...')
        time.sleep(15)
        return get_batch(symbol, interval, start_time, limit)

    if response.status_code == 200:
        return pd.DataFrame(response.json(), columns=LABELS)
    
    print(f'Got erroneous response back {symbol}: {response}. {response.text}')
    return pd.DataFrame([])


def get_candles(base, quote, start_date: datetime, end_date=None, interval='1m'):
    batches = []

    if end_date is None:
        end_date = datetime.utcnow()

    last_timestamp = int(start_date.timestamp()) * 1000
    # gather all candlesticks available, starting from the last timestamp loaded from disk or 0
    # stop if the timestamp that comes back from the api is the same as the last one
    previous_timestamp = None

    while previous_timestamp != last_timestamp:
        # stop if we reached data from today
        if datetime.fromtimestamp(last_timestamp / 1000) >= end_date:
            break

        previous_timestamp = last_timestamp

        new_batch = get_batch(
            symbol=base + quote,
            interval=interval,
            start_time=last_timestamp
        )

        # requesting candles from the future returns empty
        # also stop in case response code was not 200
        if new_batch.empty:
            break

        last_timestamp = new_batch['open_time'].max()

        # sometimes no new trades took place yet on date.today();
        # in this case the batch is nothing new
        if previous_timestamp == last_timestamp:
            break

        batches.append(new_batch)
        last_datetime = datetime.fromtimestamp(last_timestamp / 1000)

        covering_spaces = 20 * ' '
        print(datetime.now(), base, quote, interval, str(last_datetime) + covering_spaces, end='\r', flush=True)

    if len(batches) > 0:
        # write clean version of csv to parquet
        df = pd.concat(batches, ignore_index=True)
        df.drop(columns=DROP_COLUMNS, inplace=True)
        df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
        df.set_index('open_time', inplace=True)
        df = df[~df.index.duplicated(keep='first')]
        return df



In [44]:
# import requests

# url = "https://fapi.binance.com/fapi/v1/exchangeInfo"

# payload={}
# headers = {}

# response = requests.request("GET", url, headers=headers, data=payload).json()

# symbols = [s['symbol'] for s in response['symbols'] if s['contractType'] == 'PERPETUAL' and s['quoteAsset'] == 'USDT']
# # symbols[100:]
# symbols = symbols[:50]
# len(symbols)


In [45]:
symbols = [
    'BTC',
    'ETH',
    'BNB',
    'SOL',
    'ADA',
    'XRP',
    'DOT',
    'AVAX',
    'SHIB',
    'LINK',
    'TRX',
    'UNI',
    'MATIC',
    'ICP',
    'LTC',
    'NEAR',
    'APT',
    'FIL',
    'ETC',
    'ATOM',
    'WIF',
    'ARB',
    'XLM',
    'MKR',
    'OP',
    'PEPE',
    'INJ',
    'FET',
    'THETA',
    'FTM',
    'LDO',
    'SUI',
    'TIA',
    'GALA',
    'ALGO',
    'SEI',
    'AAVE',
    'BONK',
    'DYDX',
    'AEVO',
    'STRK',
    'ORDI',
    'WLD',
    'CAKE',
    '1000SATS',
    'RUNE',
    'JUP',
    'JUP',
]

In [46]:

data = []
start_time = datetime.utcnow() - timedelta(days=lookback)

for symbol in symbols:
    try:
        df = pd.read_csv(f'{DATA_PATH}{TIMEFRAME}/{symbol}-USDT.csv', parse_dates=['open_time'], index_col=['open_time'])
        df = df[df.index >= start_time].copy()
    except FileNotFoundError:
        df = pd.DataFrame([], columns=['open', 'high', 'low', 'close', 'volume'])
    # print('start time', df.index[-1].to_pydatetime())

    df1 = get_candles(
        base=symbol, 
        quote='USDT', 
        start_date=df.index[-1].to_pydatetime() if len(df) > 0 else start_time,
        # end_date=datetime.now() - timedelta(days=15),
        interval=TIMEFRAME
    )
    if df1 is not None:
        df = pd.concat([df, df1])
        df = df[~df.index.duplicated(keep='first')]

    df.rename(columns={'close': symbol}, inplace=True)
    # the data is too long, just limit to recent period
    log = np.log(df[symbol].astype(np.float32))
    data.append(log)

df = pd.concat(data, axis=1)
df = df.dropna(axis=1, how='all')
# df.dropna(inplace=True, how='any')

# our of sample
# df = df[df.index <= datetime.now() - timedelta(days=15)].copy()

df.tail()

Got erroneous response back SHIBUSDT: <Response [400]>. {"code":-1121,"msg":"Invalid symbol."}
Got erroneous response back PEPEUSDT: <Response [400]>. {"code":-1121,"msg":"Invalid symbol."}
Got erroneous response back BONKUSDT: <Response [400]>. {"code":-1121,"msg":"Invalid symbol."}


,BTC,ETH,BNB,SOL,ADA,XRP,DOT,AVAX,LINK,TRX,...,GALA,ALGO,SEI,AAVE,DYDX,AEVO,STRK,ORDI,WLD,CAKE
2024-03-30,11.151125,8.162990,6.397346,5.269666,-0.439901,-0.475298,2.244850,3.969914,2.941962,-2.098176,...,-2.695924,-1.344769,-0.200038,4.820604,1.218763,1.082144,0.759731,4.244315,2.094601,1.524512
2024-03-31,11.175535,8.202151,6.406929,5.311618,-0.429860,-0.462512,2.267787,3.991462,2.954546,-2.094515,...,-2.672201,-1.316022,-0.193099,4.855851,1.233726,1.091252,0.780013,4.249152,2.107786,1.533010
2024-04-01,11.151956,8.162379,6.356836,5.259343,-0.475298,-0.491840,2.214082,3.940300,2.911100,-2.115110,...,-2.753728,-1.379915,-0.250258,4.863681,1.182034,1.106580,0.713930,4.157147,2.037668,1.456054
2024-04-02,11.089887,8.095830,6.313240,5.202241,-0.543177,-0.534777,2.150366,3.848103,2.888704,-2.148406,...,-2.865582,-1.450300,-0.348140,4.749617,1.123630,1.125903,0.651752,4.062510,1.923577,1.367112
2024-04-03,11.101978,8.104821,6.325935,5.254308,-0.528311,-0.532901,2.165619,3.873365,2.896630,-2.141232,...,-2.849038,-1.426700,-0.320068,4.765587,1.139754,1.162526,0.668240,4.075756,1.951566,1.376446


In [47]:
p = 1
COINTEGRATION_CONFIDENCE_LEVEL = 90

# the 90%, 95%, and 99% confidence levels for the trace statistic and maximum 
# eigenvalue statistic are stored in the first, second, and third column of 
# cvt and cvm, respectively
confidence_level_cols = {
    90: 0,
    95: 1,
    99: 2
}
confidence_level_col = confidence_level_cols[COINTEGRATION_CONFIDENCE_LEVEL]


def test_johansen(symbol_pairs):
    df_t = df[symbol_pairs].copy()
    df_t.dropna(inplace=True)

    # The second and third parameters indicate constant term, with a lag of 1. 
    result = coint_johansen(df_t, 0, p)

    trace_crit_value = result.cvt[:, confidence_level_col]
    eigen_crit_value = result.cvm[:, confidence_level_col]
#     print("trace_crit_value",trace_crit_value)
#     print("eigen_crit_value",eigen_crit_value)
#     print("lr1",result.lr1)
#     print("lr2",result.lr2)

    # The trace statistic and maximum eigenvalue statistic are stored in lr1 and lr2;
    # see if they exceeded the confidence threshold
    if np.all(result.lr1 >= trace_crit_value) and np.all(result.lr2 >= eigen_crit_value):
        # print(f"{symbol_pairs} are cointegrated")
        # The first i.e. leftmost column of eigenvectors matrix, result.evec, contains the best weights.
        v1= result.evec[:,0:1]
        hr=v1/-v1[1] #to get the hedge ratio divide the best_eigenvector by the negative of the second component of best_eigenvector
        #the regression will be: close of symbList[1] = hr[0]*close of symbList[0] + error
        #where the beta of the regression is hr[0], also known as the hedge ratio, and
        #the error of the regression is the mean reverting residual signal that you need to predict, it is also known as the "spread"
        #the spread = close of symbList[1] - hr[0]*close of symbList[0] or alternatively (the same thing):
        #do a regression with close of symbList[0] as x and lose of symbList[1] as y, and take the residuals of the regression to be the spread.
        coint_pair = dict(hedge_ratio=v1[:, 0])
        for i, s in enumerate(symbol_pairs):
            coint_pair[f'sid_{i+1}'] = s

        cointegrating_pairs.append(coint_pair)



In [48]:
import traceback


def calculate_spread(df, coint_df, selected_row, hedge_ratio, nb_symbols=2):
    spread = None
    for i in range(nb_symbols):
        if spread is None:
            spread = df[coint_df[f'sid_{i + 1}'].iloc[selected_row]] * hedge_ratio[i]
        else:
            spread += df[coint_df[f'sid_{i + 1}'].iloc[selected_row]] * hedge_ratio[i]
    
    spread.dropna(inplace=True)
    return spread


critical_val = 0.02

def find_stationary_portfolio(coint_df):
    data = []
    for i, _ in coint_df.iterrows():
        try:
            hedge_ratio = coint_df.iloc[i]['hedge_ratio']
            _df = df.copy()
            spread = calculate_spread(_df, coint_df, i, hedge_ratio)
            p_val = adf_test(spread, verbose=False)
            if p_val < critical_val:
                half_life = cal_half_life(spread)
                pairs_name = coint_df[[col for col in coint_df.columns if 'sid' in col]].iloc[i].values
                # print(i, pairs_name, 'is stationary with half life', half_life)
                # print(' ')
                data.append({
                    'i': i,
                    'pairs': pairs_name,
                    'half_life': half_life
                })
        except:
            print(coint_df.iloc[i])
            # traceback.print_exc()
    
    return pd.DataFrame(data)


# find_stationary_portfolio(coint_df)

In [49]:

import itertools as it

nb_symbols = 2
cointegrating_pairs = []

#get symbol pairs
pairs = list(it.combinations(symbols, nb_symbols))

for pair in pairs:
    try:
        test_johansen(list(pair))
    except Exception:
        pass

coint_df = pd.DataFrame(cointegrating_pairs)
coint_df['i'] = coint_df.index
coint_df.head()

,hedge_ratio,sid_1,sid_2,i
0,"[3.6085735562033916, -1.8536617024518844]",BTC,AVAX,0
1,"[2.994615530781702, -1.2799850873140204]",BTC,NEAR,1
2,"[4.782194442654915, -2.2060525706998098]",ETH,AVAX,2
3,"[3.8017144235670313, -1.240606187445052]",ETH,NEAR,3
4,"[5.355801833885196, -3.1059317984363273]",BNB,UNI,4


In [50]:
coint_df.to_csv(f'coint_df_{TIMEFRAME}.csv')


# Result

In [51]:
import warnings

warnings.filterwarnings("ignore")

stationary_df = find_stationary_portfolio(coint_df)
stationary_df = pd.merge(stationary_df, coint_df, on='i')
stationary_df.dropna(inplace=True)
stationary_df['i'] = stationary_df.i.astype('int64')
stationary_df['half_life'] = stationary_df.half_life.astype('int64')
if len(stationary_df) > 0:
    stationary_df.sort_values(by=['half_life'], inplace=True)
stationary_df


,i,pairs,half_life,hedge_ratio,sid_1,sid_2
13,51,"[ICP, AEVO]",1,"[6.722148211466007, -12.416427196146783]",ICP,AEVO
22,67,"[FIL, STRK]",2,"[13.231267554081553, -14.282965407814604]",FIL,STRK
16,61,"[APT, AEVO]",2,"[8.20607911305229, -8.085188752710403]",APT,AEVO
37,96,"[AAVE, CAKE]",4,"[12.936464934198135, -7.867127836851941]",AAVE,CAKE
28,81,"[INJ, ORDI]",6,"[5.182645576499397, -4.929944454336126]",INJ,ORDI
27,79,"[OP, ORDI]",7,"[4.7853717788470655, -4.6190140750082564]",OP,ORDI
26,78,"[XLM, CAKE]",7,"[15.730272707598852, -5.536364850394238]",XLM,CAKE
38,97,"[DYDX, ORDI]",9,"[3.8595963137795306, 1.945165084615886]",DYDX,ORDI
31,84,"[THETA, STRK]",9,"[3.378631770713758, -0.04679575110359926]",THETA,STRK
34,88,"[TIA, ORDI]",11,"[3.26660393391845, -1.120104099219162]",TIA,ORDI


In [52]:
stationary_df.to_csv(f'stationary_df_{TIMEFRAME}.csv', index=False)
# stationary_df

In [55]:
ignored_symbols = ['ETC', 'DOT', 'NEO', 'NEAR']
ignored_symbols= []
mask = ~(stationary_df['sid_1'].isin(ignored_symbols) | stationary_df['sid_2'].isin(ignored_symbols))
filtered_df = stationary_df[mask]
# filtered_df['hdiff'] = filtered_df['hedge_ratio'].apply(lambda x: np.sum(x))
# filtered_df.query('abs(hdiff) <= 10')

# add 1 column tradingview_symbol that combine: hedge_ratio[0] * str(sid_1 + 'USDT.P') - str(sid_2 + 'USDT.P') * hedge_ratio[1]
filtered_df['tradingview_symbol'] = filtered_df.apply(lambda x: f'{x.sid_1}USDT.P * {x.hedge_ratio[0]} - {x.sid_2}USDT.P * {x.hedge_ratio[1]}', axis=1)

filtered_df.to_csv('pairs.csv', index=False)